In [1]:
import geopandas as gpd
from shapely.geometry import Point

gdf = gpd.read_file('Results/U2018_CLC2018_V2020_20u1.json')


In [3]:
print(gdf.columns)

Index(['OBJECTID', 'Code_18', 'Remark', 'Area_Ha', 'ID', 'Shape_Length',
       'Shape_Area', 'geometry'],
      dtype='object')


In [5]:
print(gdf.iloc[0]['Code_18'])
print(gdf.iloc[0]['OBJECTID'])
print(gdf.iloc[0]['Remark'])
print(gdf.iloc[0]['ID'])

111
111
None
EU_111


In [6]:
coord = Point(-8.5772292, 37.283231)  


for index, row in gdf.iterrows():
    if row['geometry'].contains(coord):
        print(row['Code_18'])
        break

324


In [11]:
clc_codes = []
for i in range(len(df)):
    coord = Point(df.iloc[i]['longitude'], df.iloc[i]['latitude'])
    found = False
    for index, row in gdf.iterrows():
        if row['geometry'].contains(coord):
            print(f"{i} : {row['Code_18']}")
            clc_codes.append(row['Code_18'])
            found = True
            break
    if not found:
        clc_codes.append(None)

    # Save the DataFrame every 100 iterations
    if i % 100 == 0:
        temp_df = df.iloc[:i+1].copy()  # Create a new DataFrame that only includes rows up to the current iteration
        temp_df['CLC_CODE'] = clc_codes
        temp_df.to_csv('NaturalFiresTOPOGRAPHY.csv', index=False)

# Save the DataFrame one last time at the end to make sure all changes are saved
df['CLC_CODE'] = clc_codes
df.to_csv('NaturalFiresTOPOGRAPHY.csv', index=False)

0 : 324
1 : 142
2 : 112
3 : 242
4 : 243
5 : 242
6 : 241
7 : 512
8 : 121
9 : 243
10 : 242
11 : 242
12 : 242
13 : 112
14 : 242
15 : 322
16 : 112
17 : 242
18 : 241
19 : 242
20 : 324
21 : 313
22 : 243
23 : 323
24 : 243
25 : 241
26 : 242
27 : 243
28 : 242
29 : 242
30 : 322
31 : 322
32 : 242
33 : 241
34 : 211
35 : 241
36 : 242
37 : 211
38 : 111
39 : 241
40 : 242
41 : 242
42 : 211
43 : 242
44 : 243
45 : 322
46 : 241
47 : 313
48 : 241
49 : 211
50 : 112
51 : 242
52 : 211
53 : 122
54 : 323
55 : 242
56 : 242
57 : 231
58 : 244
59 : 324
60 : 241
61 : 112
62 : 112
63 : 223
64 : 324
65 : 244
66 : 244
67 : 112
68 : 111
69 : 112
70 : 131
71 : 111
72 : 313
73 : 242
74 : 242
75 : 242
76 : 211
77 : 211
78 : 112
79 : 112
80 : 322
81 : 313
82 : 241
83 : 212
84 : 324
85 : 241
86 : 322
87 : 243
88 : 512
89 : 242
90 : 213
91 : 112
92 : 324
93 : 212
94 : 112
95 : 223
96 : 111
97 : 324
98 : 324
99 : 324
100 : 324
101 : 242
102 : 242
103 : 242
104 : 243
105 : 211
106 : 221
107 : 243
108 : 243
109 : 211
110 : 211


In [7]:
from geopandas.tools import sjoin
import geopandas
import pandas as pd

df = pd.read_csv('LocationsWFire.csv')

# Create a new GeoDataFrame from df that includes a 'geometry' column with the coordinates
points = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))

# Create a spatial index for gdf
gdf.sindex

# Print out the columns of gdf
print("Columns in gdf: ", gdf.columns)

# Use spatial join to find the polygons that contain each point
joined = sjoin(points, gdf, how='left', op='within')

# Print out the columns of joined
print("Columns in joined: ", joined.columns)

# The 'Code_18' values for each point are now in the 'Code_18_right' column of 'joined'
df['CLC_CODE'] = joined['Code_18']

df.to_csv('LocationsWFire.csv', index=False)

Columns in gdf:  Index(['OBJECTID', 'Code_18', 'Remark', 'Area_Ha', 'ID', 'Shape_Length',
       'Shape_Area', 'geometry'],
      dtype='object')
Columns in joined:  Index(['date', 'latitude', 'longitude', 'slope', 'roughness', 'aspect',
       'geometry', 'index_right', 'OBJECTID', 'Code_18', 'Remark', 'Area_Ha',
       'ID', 'Shape_Length', 'Shape_Area'],
      dtype='object')


/home/ori/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_17092/1911711095.py:17: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = sjoin(points, gdf, how='left', op='within')


In [ ]:
import pandas as pd

df = pd.read_csv('LocationsWFire.csv')

clc_err = [-1]

for i in range(len(df)):
    date = df.iloc[i]['date']
    #district = df.iloc[i]['district']
    #municipality = df.iloc[i]['municipality']
    #parish = df.iloc[i]['parish']
    #local = df.iloc[i]['local']
    latitude = float(df.iloc[i]['latitude'])
    longitude = float(df.iloc[i]['longitude'])


    clc_code = df.iloc[i]['CLC_CODE']
    if pd.isna(clc_code):
        clc_code = -1
    else:
        clc_code = int(clc_code)

    #if(clc_code in clc_err):
    #        print(f"{i} : {clc_code}")
    #        #print(f"{date},{district},{municipality},{parish},{local},{latitude},{longitude}")
    #        print(f"{date},{latitude},{longitude}")
    #        continue
    
    while(clc_code in clc_err):
            latitude += 0.001
            longitude += 0.001
            coord = Point(longitude, latitude)  
            for index, row in gdf.iterrows():
                if row['geometry'].contains(coord):
                    new_clc_code = row['Code_18']
                    break

            if pd.isna(new_clc_code):
                new_clc_code = -1
            else:
                new_clc_code = int(new_clc_code)

            if new_clc_code not in clc_err:
                clc_code = new_clc_code
                updated = True

    if(updated):
            print(f"{i} : {clc_code}")
            df.at[i, 'CLC_CODE'] = clc_code
            updated = False


    '''
    if(int(date.split("-")[0]) < 2013):
        
        try:
            clc_code = int(df.iloc[i]['CLC_CODE'])
        except:
            clc_code = -1
    
        while(clc_code in clc_err):
            latitude += 0.0001
            longitude += 0.0001
            coord = Point(longitude, latitude)  
            for index, row in gdf.iterrows():
                if row['geometry'].contains(coord):
                    #print(row['Code_18'])
                    clc_code = row['Code_18']
                    break
                    
        print(f"{i} : {clc_code}")
        df.at[i, 'CLC_CODE'] = clc_code
        '''

            

KeyboardInterrupt: 

In [10]:
from shapely.geometry import Point
from geopandas.tools import sjoin
import geopandas
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

gdf.sindex

df = pd.read_csv('LocationsWFire.csv')

clc_err = [-1]

for i in range(len(df)):
    date = df.iloc[i]['date']
    #district = df.iloc[i]['district']
    #municipality = df.iloc[i]['municipality']
    #parish = df.iloc[i]['parish']
    #local = df.iloc[i]['local']
    latitude = float(df.iloc[i]['latitude'])
    longitude = float(df.iloc[i]['longitude'])
    updated = False

    
    clc_code = df.iloc[i]['CLC_CODE']
    if pd.isna(clc_code):
        clc_code = -1
    else:
        clc_code = int(clc_code)

    
    if(clc_code in clc_err):
            print(f"{i} : {clc_code}")
            #print(f"{date},{district},{municipality},{parish},{local},{latitude},{longitude}")
            print(f"{date},{latitude},{longitude}")
            continue
    


    '''
    max_iterations = 1000  # Set a limit to the number of iterations
    iteration = 0

    while(clc_code in clc_err and iteration < max_iterations):
        latitude += 0.001
        longitude += 0.001
        coord = Point(longitude, latitude)  
        point = geopandas.GeoDataFrame([{'geometry': coord}])
        joined = sjoin(point, gdf, how='left', op='within')
        if joined.empty:
            break
        
        new_clc_code = joined.iloc[0]['Code_18']
        if pd.isna(new_clc_code):
            new_clc_code = -1
        else:
            new_clc_code = int(new_clc_code)
        if new_clc_code not in clc_err:
            clc_code = new_clc_code
            updated = True

        iteration += 1

    if(updated):
        print(f"{i} : {clc_code}")
        df.at[i, 'CLC_CODE'] = clc_code
        updated = False
    '''
    

172 : -1
2006-7-13,37.17454267618904,-7.401703836851796


In [8]:
df.to_csv('LocationsWFire.csv', index=False)

In [3]:
import pandas as pd
df = pd.read_csv('NaturalFiresTOPOGRAPHY_mean_elev.csv')

for i in range(len(df)):
    #year,date,district,municipality,parish,local,latitude,longitude
    year = df.iloc[i]['year']
    date = df.iloc[i]['date']
    district = df.iloc[i]['district']
    municipality = df.iloc[i]['municipality']
    parish = df.iloc[i]['parish']
    local = df.iloc[i]['local']
    lat = df.iloc[i]['latitude']
    lon = df.iloc[i]['longitude']
    clc = df.iloc[i]['CLC_CODE']

    if(int(clc) > 400):

        print(f"{year},{date},{district},{municipality},{parish},{local},{lat},{lon},{clc}")



2013,2013-09-04,Coimbra,Pampilhosa da Serra,Vidual,Vidual de Baixo,40.0939339998033,-7.85871599992116,512.0


In [6]:
#,,,,,,,,,,,

#40.0926491,-7.8620043
40.0939339998033,-7.85871599992116
coord = Point(-7.8620043, 40.0926491)  
for index, row in gdf.iterrows():
    if row['geometry'].contains(coord):
        print(row['Code_18'])
        break

332
